For a subset of pulsars with known binary companions, calculate roc curves to determine the ideal search radius to be performing the query in

In [1]:
import astropy
import astropy.units as u
import astroquery
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import quantity_support
from astropy.time import Time 
import os
from astropy.io.votable import parse_single_table
from astropy.time import Time
import pytest
from astropy.table import Table, vstack
import csv
import pandas as pd
from astropy.io.votable import from_table, writeto

In [2]:
# create astropy table superset of all identified 'hits' from matches_10arcsec.npz
load = np.load('/home/billee/Binary-Pulsar-Distances/Binary_Pulsar_Distances/matches_10arcsec.npz')
superset = Table([load['Pulsar'], load['Pulsar_RA'], load['Pulsar_DEC'], load['Binary'], load['Bin_Comp'], 
                  load['ra'], load['dec']], names= ['Pulsar', 'Pulsar RA', 'Pulsar DEC', 'Binary', 'Bin Comp',
                                                     'ra', 'dec'])

In [7]:
# create a subset of 'superset' that contains just pulsars known (according to ATNF) to be in binaries

subset = superset
l = 0
for k in subset['Binary']:
    if k == '*':
        subset.remove_row(l)
    else:
        l += 1

In [8]:
# add to this new subset table, separations of each pulsar from the given Gaia source

# SkyCoord for each pulsar and Gaia source, respectively
psr_pos = SkyCoord(subset['Pulsar RA'], subset['Pulsar DEC'], frame = 'icrs', unit =(u.hourangle, u.deg))
source_pos = SkyCoord(subset['ra'], subset['dec'], frame = 'icrs', unit= (u.deg, u.deg))

source_seps = []
first_time = True
# this for loop will create a table of the sky separations fo each pulsar and Gaia source pair in the astropy table
for k in range(len(subset)):
    if first_time:
        source_seps.insert(0,psr_pos[k].separation(source_pos[k]))
        first_time = False
    else:
        source_seps.append(psr_pos[k].separation(source_pos[k]))

subset.add_column(source_seps, name = 'Separation')

In [9]:
# make an array of ranges of radii to test -- from 0 to 10 arcsec
radii = np.linspace(0,10,100)

In [11]:
# calculate probability of detection (pd) and probability of false alarm (pfa)

total_psrs = 1
n_hits_true = 1
n_hits_false = 1

pd = n_hits_true / total_psrs
pfa = n_hits_false / total_psrs

#calculate the number of total individual pulsars (no reduplication)
current = 'J0024-7204E'
psrs_with_hits = [current] # total number of PULSARS with hits, distinct from total number of hits
pwh_binary = ['DD']
for b in range(len(subset['Pulsar'])):
    if subset['Pulsar'][b] != current:
        psrs_with_hits.append(subset['Pulsar'][b])
        pwh_binary.append(subset['Binary'][b])
        current = subset['Pulsar'][b] 

In [12]:
# toy model with a small enough separation that it is easy to go in by hand to do:
subset[400:415]

Pulsar,Pulsar RA,Pulsar DEC,ra,dec,Binary,Separation
,,,,,,deg
str12,str16,str16,str18,str19,str6,float64
J0024-7204Y,00:24:01.4026,-72:04:41.8363,6.001653822714394,-72.08071400891852,ELL1,0.0027474776346458223
J0337+1715,03:37:43.82589,+17:15:14.828,54.43261340914772,17.254115461920154,ELL1,6.29892323013655e-06
J0348+0432,03:48:43.639000,+04:32:11.4580,57.18183331779655,4.5365161957787254,ELL1,4.1389910062538866e-06
J0437-4715,04:37:15.8961737,-47:15:09.110714,69.31662758499185,-47.25268764316094,T2,0.00030978067618349044
J0514-4002A,05:14:06.69271,-40:02:48.8930,78.52799552909461,-40.04700266794965,DDH,0.00012135601305043101
J0514-4002A,05:14:06.69271,-40:02:48.8930,78.5280939738137,-40.046497857750886,DDH,0.00044615264121269373
J0514-4002A,05:14:06.69271,-40:02:48.8930,78.52753203005061,-40.04733050061802,DDH,0.0004964042623038699
J0514-4002A,05:14:06.69271,-40:02:48.8930,78.52797467644648,-40.0464105346105,DDH,0.0005087072581887858
